<a href="https://colab.research.google.com/github/gvogiatzis/trafficsim/blob/main/suptrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/gvogiatzis/trafficsim.git

In [ ]:
cd trafficsim

In [ ]:
!wget "https://7d9c-80-195-93-221.ngrok-free.app/sumo_data/flowmat.txt"

In [ ]:
from rl import SupervisedLearningPretrainer

import matplotlib.pyplot as plt
import torch
import tqdm
import numpy as np

from rl.models import MLPnet, loadModel, saveModel, loadModel_from_dict
import matplotlib.pyplot as plt

num_epochs = 50
network_layers = "1024x1024"
dataset_size = 100000
network_layers = [int(s) for s in network_layers.split("x") if s.isnumeric()]
out_model_fname = "sup_model.pt"
flowmat_fname = "flowmat.txt"

In [ ]:
dataset = []
W = np.loadtxt(flowmat_fname)

output_dim, input_dim = W.shape
Wtorch = torch.tensor(W,dtype=torch.float32)
print("Generating dataset:")
for i in tqdm.tqdm(range(dataset_size)):
    x = 50*torch.randn(size=(input_dim,), dtype=torch.float32)
    a = np.argsort(Wtorch @ x)
    t = a[0]
    dataset.append((x, t))
model = MLPnet(input_dim,*network_layers,output_dim)
trainer = SupervisedLearningPretrainer(dataset, model,use_gpu=True)

stats = trainer.train_epochs(num_epochs)

saveModel(model, out_model_fname)
plt.figure()
plt.title("Loss")
plt.plot(stats["training_loss_series"], 'r-')
plt.plot(stats["test_loss_series"], 'b-')
plt.figure()
plt.title("Accuracy")
plt.plot(stats["training_acc_series"], 'r-')
plt.plot(stats["test_acc_series"], 'b-')
plt.show()